<h3>Imports and setup</h3>

In [36]:
from pathlib import Path
import os
import tifffile
import czifile
from tqdm import tqdm
import napari
import pandas as pd
import numpy as np


<h3>Image data scraping</h3>

In [37]:
# Define the directory containing the raw .czi files and annotations
directory_path = Path("./apeer_annotations/")

# Initialize an empty list to store the .czi and ome.tiff annotation filepaths
czi_images = []
annotations = []

# Iterate through the files in the directory
for file_path in directory_path.glob('*.czi'):
    # Append the file to the corresponding group
    czi_images.append(str(file_path))

# Iterate through the files in the directory
for file_path in directory_path.glob('*.tiff'):
    # Append the file to the corresponding group
    annotations.append(str(file_path))


<h3>Transforming .czi files into .tiff</h3>

In [57]:
# Define output directory for .tiff converted .czi files  
processed_czi_dir = "./processed_bf_organoids/train_images"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_czi_dir)
    print(f"Output folder created: {processed_czi_dir}")
except FileExistsError:
    print(f"Output folder already exists: {processed_czi_dir}")
    
for filepath in tqdm(czi_images):
    # Get the filename without the extension
    filename = Path(filepath).stem

    # Read the .czi file and remove singleton dimensions
    img = czifile.imread(filepath)
    img = img.squeeze()

    # Save the image as a regular .tiff file under processed_bf_organoids/train_images
    tifffile.imwrite(os.path.join(processed_czi_dir, f"{filename}.tiff"), img)


Output folder created: ./processed_bf_organoids/train_images


100%|██████████| 119/119 [00:01<00:00, 80.41it/s]


<h3>Rename annotations and create a copy under preprocessed_annotations</h3>

In [58]:
# Define output directory for renamed annotations
processed_annotations = "./preprocessed_annotations"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_annotations)
    print(f"Output folder created: {processed_annotations}")
except FileExistsError:
    print(f"Output folder already exists: {processed_annotations}")
    
for filepath in tqdm(annotations):
    # Get the filename without the extension
    filename = Path(filepath).stem
    # Remove _finalprediction.ome so it matches the original image names
    filename = filename.replace("_finalprediction.ome", "")
    
    # Read the ome.tiff file, ignore ome
    img = tifffile.imread(filepath, is_ome=False)
    
    # Save the image as a regular .tiff file under preprocessed_annotations
    tifffile.imwrite(os.path.join(processed_annotations, f"{filename}.tiff"), img)

Output folder created: ./preprocessed_annotations


100%|██████████| 119/119 [00:01<00:00, 79.30it/s]


<h3>Process organoid annotations into organoid_train_masks</h3>

In [85]:
# Define the directory containing the renamed annotation files
prep_annotation_path = Path("./preprocessed_annotations/")

# Initialize an empty list to store the renamed annotation filepaths
preprocessed_annotations = []

# Iterate through the files in the directory
for file_path in prep_annotation_path.glob('*.tiff'):
    # Append the file to the corresponding group
    preprocessed_annotations.append(str(file_path))

# Define output directory for processsed dead_organoid annotations
processed_dead_annotations = "./processed_bf_organoids/train_masks/dead"
processed_diff_annotations = "./processed_bf_organoids/train_masks/differentiated"
processed_undiff_annotations = "./processed_bf_organoids/train_masks/undifferentiated"

# Store the directories in a list to loop through them
proc_ann_directories = [processed_dead_annotations, processed_diff_annotations, processed_undiff_annotations]

# Check if the output folders exists, create it if not
for directory in proc_ann_directories:
        
    try:
        os.makedirs(directory)
        print(f"Output folder created: {directory}")
    except FileExistsError:
        print(f"Output folder already exists: {directory}")
    
# Loop trough each .tiff containing the class annotations
for filepath in tqdm(preprocessed_annotations):
    
    # Get the filename without the extension
    filename = Path(filepath).stem
    
    # Read the .tiff containing all 3 class annotations
    img = tifffile.imread(filepath, is_ome=False)

    # Extract the channel dimension containing each class annotation
    dead_organoids = img[0,:,:]
    diff_organoids = img[1,:,:]
    undiff_organoids = img[2,:,:]

    # Binarize the arrays
    dead_organoids = np.where(dead_organoids !=0, 255, 0)
    diff_organoids = np.where(diff_organoids !=0, 255, 0)
    undiff_organoids = np.where(undiff_organoids !=0, 255, 0)

    # Convert the array to uint8
    dead_organoids = dead_organoids.astype(np.uint8)
    diff_organoids = diff_organoids.astype(np.uint8)
    undiff_organoids = undiff_organoids.astype(np.uint8)

    # Store the resulting array as a .tiff file under train masks
    tifffile.imwrite(os.path.join(processed_dead_annotations, f"{filename}.tiff"), dead_organoids)
    tifffile.imwrite(os.path.join(processed_diff_annotations, f"{filename}.tiff"), diff_organoids)
    tifffile.imwrite(os.path.join(processed_undiff_annotations, f"{filename}.tiff"), undiff_organoids)

Output folder already exists: ./processed_bf_organoids/train_masks/dead
Output folder created: ./processed_bf_organoids/train_masks/differentiated
Output folder created: ./processed_bf_organoids/train_masks/undifferentiated


100%|██████████| 119/119 [00:02<00:00, 44.71it/s]


<h3>Visualization</h3>

In [101]:
# Define below the filename you want to display together with the corresponding classes
file = "MIN_P1A01.tiff"

# Initialize Napari instance to display raw_images together with class masks
viewer = napari.Viewer(ndisplay=2)

# Read raw image and class labels to display
raw_img = tifffile.imread(f"./processed_bf_organoids/train_images/{file}", is_ome=False)
dead_mask = tifffile.imread(f"./processed_bf_organoids/train_masks/dead/{file}", is_ome=False)
diff_mask =  tifffile.imread(f"./processed_bf_organoids/train_masks/differentiated/{file}", is_ome=False)
undiff_mask = tifffile.imread(f"./processed_bf_organoids/train_masks/undifferentiated/{file}", is_ome=False)

# Display in Napari
viewer.add_image(raw_img, name=f"Raw {file}")
viewer.add_labels(dead_mask, name=f"Dead {file}")
viewer.add_labels(diff_mask, name=f"Differentiated {file}")
viewer.add_labels(undiff_mask, name=f"Spheroid {file}")

<Labels layer 'Spheroid MIN_P1A01.tiff' at 0x16607a8f5b0>

<h3>Data quality checks</h3>

In [53]:
# Read measurements_default.csv containing each object_id's class_name
df = pd.read_csv("./apeer_annotations/measurements_default.csv")